In [1]:
# -*- coding: utf-8 -*-
# PMT Diffusion Transformer (DiT-style) for 5160 PMTs
# Inputs per PMT: [npe, time] + [xpmt, ypmt, zpmt]
# Output: predicted noise for [npe, time]
#
# Model:
#   - Signal MLP: (2)  -> (h)
#   - Position MLP: (3) -> (h)
#   - Sum to form token embeddings: (B, 5160, h)
#   - DiT-style conditioning: timestep + condition (6-dim) via AdaLN-Zero in each block
#   - Self-attention over 5160 tokens
#   - Head: (h) -> (2) per token
#
# Notes:
#   - Dataset expected to yield: {"x":[B,2,5160], "pos":[B,5160,3], "c":[B,6]}
#   - Training step shows simple VP-style noising for demo.

import math
import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from typing import Optional, Dict


# ----------------------------
# Dataset (same contract as discussed)
# ----------------------------
def _np_stack3(x1, x2, x3):
    import numpy as np
    a = np.asarray(x1).reshape(-1)
    b = np.asarray(x2).reshape(-1)
    c = np.asarray(x3).reshape(-1)
    return np.stack([a, b, c], axis=-1)  # (L,3)

class PMTH5Dataset(Dataset):
    """
    Event-wise loader for an HDF5 file with:
      - /input:     (N, 2, 5160)    charge & time
      - /label:     (N, 6)          (we will return it as condition 'c')
      - /xpmt:      (5160,)
      - /ypmt:      (5160,)
      - /zpmt:      (5160,)
    """
    def __init__(self, h5_path: str,
                 input_key: str="input", condition_key: str="label",
                 x_key: str="xpmt", y_key: str="ypmt", z_key: str="zpmt",
                 dtype: torch.dtype=torch.float32):
        super().__init__()
        self.h5_path = h5_path
        self.input_key = input_key
        self.condition_key = condition_key
        self.x_key = x_key
        self.y_key = y_key
        self.z_key = z_key
        self.dtype = dtype

        self._h5: Optional[h5py.File] = None
        self._input = None
        self._cond  = None
        self._xp    = None
        self._yp    = None
        self._zp    = None
        self._length = None

    def _ensure_open(self):
        if self._h5 is None:
            self._h5 = h5py.File(self.h5_path, "r")
            self._input = self._h5[self.input_key]
            self._cond  = self._h5[self.condition_key]
            self._xp    = self._h5[self.x_key]
            self._yp    = self._h5[self.y_key]
            self._zp    = self._h5[self.z_key]
            self._length = self._input.shape[0]

    def __len__(self):
        self._ensure_open()
        return self._length

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        self._ensure_open()
        x_np  = self._input[idx]                            # (2,5160)
        c_np  = self._cond[idx]                             # (6,)
        pos_np = _np_stack3(self._xp, self._yp, self._zp)   # (5160,3)

        x   = torch.as_tensor(x_np, dtype=self.dtype)               # (2,5160)
        pos = torch.as_tensor(pos_np, dtype=self.dtype)             # (5160,3)
        c   = torch.as_tensor(c_np, dtype=self.dtype)               # (6,)
        return {"x": x, "pos": pos, "c": c, "idx": idx}


# ----------------------------
# Utils: embeddings and AdaLN-Zero
# ----------------------------
def sinusoidal_timestep_embedding(t: torch.Tensor, dim: int) -> torch.Tensor:
    """
    Create sinusoidal timestep embeddings (as in Transformer pos-emb).
    t: (B,) in [0, 1] or integer timesteps scaled.
    Returns (B, dim)
    """
    device = t.device
    half = dim // 2
    # Use log-scale frequencies.
    freqs = torch.exp(
        torch.linspace(
            math.log(1.0), math.log(10000.0), steps=half, device=device
        )
    )
    args = t[:, None] * freqs[None, :]
    emb = torch.cat([torch.sin(args), torch.cos(args)], dim=-1)
    if dim % 2 == 1:  # pad if odd
        emb = torch.nn.functional.pad(emb, (0,1))
    return emb  # (B, dim)

class MLP(nn.Module):
    """Simple 2-layer MLP with GELU."""
    def __init__(self, in_dim, hidden, out_dim, dropout=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, out_dim),
        )
    def forward(self, x):  # (..., in_dim) -> (..., out_dim)
        return self.net(x)

class AdaLNZero(nn.Module):
    """
    AdaLN-Zero modulation block (as in DiT).
    Given a normalized hidden h and conditioning vector m, produce:
      mod(h) = h * (1 + gamma) + beta
    where [gamma, beta] are predicted from m. Optionally use a residual gate.
    """
    def __init__(self, hidden_dim: int, cond_dim: int):
        super().__init__()
        self.ln = nn.LayerNorm(hidden_dim, elementwise_affine=False)
        # Predict scale, shift, and residual gate from cond vector.
        self.mod = nn.Linear(cond_dim, hidden_dim * 3)
        # Initialize to zeros for "Zero" init (stabilizes early training).
        nn.init.zeros_(self.mod.weight)
        nn.init.zeros_(self.mod.bias)

    def forward(self, x: torch.Tensor, cond_vec: torch.Tensor):
        """
        x: (B, L, H)
        cond_vec: (B, cond_dim)
        Returns:
           x_mod: (B, L, H)
           gate:  (B, 1, H) gate for residual path
        """
        h = self.ln(x)
        m = self.mod(cond_vec)  # (B, 3H)
        gamma, beta, gate = torch.chunk(m, 3, dim=-1)  # (B,H) each
        gamma = gamma.unsqueeze(1)
        beta  = beta.unsqueeze(1)
        gate  = gate.unsqueeze(1)
        x_mod = h * (1 + gamma) + beta
        return x_mod, gate


# ----------------------------
# Transformer Block with AdaLN
# ----------------------------
class DiTBlock(nn.Module):
    """
    Transformer block with AdaLN-Zero conditioning for both Attn and MLP paths.
    """
    def __init__(self, hidden_dim: int, n_heads: int, mlp_ratio: float=4.0, dropout: float=0.0, cond_dim: int=256):
        super().__init__()
        self.attn_mod = AdaLNZero(hidden_dim, cond_dim)
        self.mlp_mod  = AdaLNZero(hidden_dim, cond_dim)

        self.attn = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=n_heads,
                                          batch_first=True, dropout=dropout)
        self.attn_drop = nn.Dropout(dropout)

        mlp_hidden = int(hidden_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden, hidden_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor, cond_vec: torch.Tensor):
        # Attention path
        x_mod, gate_attn = self.attn_mod(x, cond_vec)     # (B,L,H), (B,1,H)
        attn_out, _ = self.attn(x_mod, x_mod, x_mod, need_weights=False)
        x = x + gate_attn * self.attn_drop(attn_out)

        # MLP path
        x_mod, gate_mlp = self.mlp_mod(x, cond_vec)
        x = x + gate_mlp * self.mlp(x_mod)
        return x


# ----------------------------
# PMT DiT Model
# ----------------------------
class PMTDiT(nn.Module):
    """
    PMT DiT-style transformer for diffusion noise prediction.
    - signal_mlp: (npe,time)->h
    - pos_mlp:    (xpmt,ypmt,zpmt)->h
    - token = signal_emb + pos_emb
    - K transformer blocks with AdaLN-Zero conditioned on (timestep + condition)
    - head projects tokens to 2-dim noise per PMT
    """
    def __init__(self,
                 hidden_dim: int=256,
                 depth: int=6,
                 n_heads: int=8,
                 cond_in_dim: int=6,      # condition vector dimension
                 t_embed_dim: int=256,    # timestep embedding dimension
                 cond_embed_dim: int=256, # condition embedding dimension
                 dropout: float=0.0):
        super().__init__()
        self.hidden_dim = hidden_dim

        # Embedding blocks
        self.signal_mlp = MLP(in_dim=2, hidden=hidden_dim, out_dim=hidden_dim, dropout=dropout)
        self.pos_mlp    = MLP(in_dim=3, hidden=hidden_dim, out_dim=hidden_dim, dropout=dropout)

        # Timestep and condition embeddings
        self.t_embed_dim = t_embed_dim
        self.c_embed = MLP(in_dim=cond_in_dim, hidden=cond_embed_dim, out_dim=t_embed_dim, dropout=dropout)

        # Fuse t and c -> global cond vector for blocks
        self.fuse = MLP(in_dim=t_embed_dim, hidden=t_embed_dim, out_dim=t_embed_dim, dropout=dropout)

        # Transformer blocks
        self.blocks = nn.ModuleList([
            DiTBlock(hidden_dim=hidden_dim, n_heads=n_heads, mlp_ratio=4.0, dropout=dropout, cond_dim=t_embed_dim)
            for _ in range(depth)
        ])

        # Output head
        self.norm = nn.LayerNorm(hidden_dim)
        self.head = nn.Linear(hidden_dim, 2)  # predict noise for (npe, time)

    def forward(self, x_sig: torch.Tensor, pos: torch.Tensor, c: torch.Tensor, t: torch.Tensor):
        """
        x_sig: (B, 2, 5160)          signal [npe, time]
        pos:   (B, 5160, 3)          positions [x,y,z]
        c:     (B, 6)                condition
        t:     (B,)                  timestep in [0,1] (float) or scaled ints / T
        Returns:
          eps_pred: (B, 2, 5160)
        """
        B = x_sig.shape[0]
        # Prepare tokens: (B, 5160, h)
        sig_tok = self.signal_mlp(x_sig.transpose(1, 2))  # (B,5160,2)->(B,5160,h)
        pos_tok = self.pos_mlp(pos)                       # (B,5160,3)->(B,5160,h)
        tok = sig_tok + pos_tok                           # (B,5160,h)

        # Build DiT conditioning vector
        t_emb = sinusoidal_timestep_embedding(t, self.t_embed_dim)  # (B, D)
        c_emb = self.c_embed(c)                                     # (B, D)
        cond_vec = self.fuse(t_emb + c_emb)                         # (B, D)

        # Transformer
        h = tok
        for blk in self.blocks:
            h = blk(h, cond_vec)                                    # (B,5160,h)

        h = self.norm(h)
        eps = self.head(h)            # (B,5160,2)
        return eps.transpose(1, 2)    # (B,2,5160)


# ----------------------------
# Simple training step (demo)
# ----------------------------
@torch.no_grad()
def vp_noising(x0: torch.Tensor, t: torch.Tensor):
    """
    Simple VP-style noising: x_t = sqrt(1 - t) * x0 + sqrt(t) * eps
    x0: (B,2,5160), t: (B,) in [0,1]
    Returns: x_t, eps
    """
    B = x0.shape[0]
    eps = torch.randn_like(x0)
    t = t.view(B, 1, 1)
    x_t = torch.sqrt(1.0 - t) * x0 + torch.sqrt(t) * eps
    return x_t, eps

def training_step(model: PMTDiT, batch: Dict[str, torch.Tensor], optimizer, device):
    """
    One training step:
      - Sample t ~ U(0,1)
      - Make x_t and eps
      - Predict eps and compute MSE
    """
    model.train()
    x0  = batch["x"].to(device)        # (B,2,5160)
    pos = batch["pos"].to(device)      # (B,5160,3)
    c   = batch["c"].to(device)        # (B,6)
    B   = x0.shape[0]
    t   = torch.rand(B, device=device) # (B,)

    with torch.no_grad():
        x_t, eps = vp_noising(x0, t)

    eps_pred = model(x_sig=x_t, pos=pos, c=c, t=t)  # (B,2,5160)

    loss = torch.mean((eps_pred - eps) ** 2)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    return loss.item()


# ----------------------------
# Minimal usage example
# ----------------------------
if __name__ == "__main__":
    # Adjust paths and hyperparams as needed.
    h5_path = "/home/work/GENESIS/GENESIS-data/22644_0921.h5"
    batch_size = 2
    device = "cuda" if torch.cuda.is_available() else "cpu"

    ds = PMTH5Dataset(h5_path=h5_path)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    model = PMTDiT(hidden_dim=256, depth=4, n_heads=8, cond_in_dim=6, t_embed_dim=256, cond_embed_dim=256, dropout=0.0).to(device)
    optim = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)

    # One demo step
    batch = next(iter(loader))
    loss = training_step(model, batch, optim, device)
    print(f"[demo] loss: {loss:.6f}")


[demo] loss: 1.220773


In [2]:
# -*- coding: utf-8 -*-
# PMT Diffusion Transformer (DiT-style) for 5160 PMTs with DDPM q-sample
# Inputs per PMT: [npe, time] + [xpmt, ypmt, zpmt]
# Output: predicted noise for [npe, time]
#
# Dataset yields:
#   x:   (B, 2, 5160)     # [npe, time]
#   pos: (B, 5160, 3)     # [xpmt, ypmt, zpmt]
#   c:   (B, 6)           # condition vector (from '/label')
#
# Model:
#   - Signal MLP:   (2)  -> (h)
#   - Position MLP: (3)  -> (h)
#   - Token = signal_emb + pos_emb  => (B, 5160, h)
#   - K DiT-style transformer blocks with AdaLN-Zero conditioning
#   - Head: (h) -> (2) per token   => (B, 2, 5160)
#
# Training:
#   - DDPM q-sample with linear or cosine beta schedule
#   - t_int ~ Unif{0..T-1}, x_t = sqrt(alpha_bar[t])*x0 + sqrt(1-alpha_bar[t])*eps
#   - Model predicts eps, MSE loss

import math
from typing import Optional, Dict

import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


# ----------------------------
# Dataset
# ----------------------------
def _np_stack3(x1, x2, x3):
    """Stack 1D arrays into shape (L, 3)."""
    a = np.asarray(x1).reshape(-1)
    b = np.asarray(x2).reshape(-1)
    c = np.asarray(x3).reshape(-1)
    return np.stack([a, b, c], axis=-1)  # (L,3)

class PMTH5Dataset(Dataset):
    """
    Event-wise loader for an HDF5 file with:
      - /input: (N, 2, 5160)   charge & time
      - /label: (N, 6)         condition vector (returned as 'c')
      - /xpmt, /ypmt, /zpmt: (5160,) PMT positions
    """
    def __init__(self, h5_path: str,
                 input_key: str = "input",
                 condition_key: str = "label",  # read 'label' but return as 'c'
                 x_key: str = "xpmt",
                 y_key: str = "ypmt",
                 z_key: str = "zpmt",
                 dtype: torch.dtype = torch.float32):
        super().__init__()
        self.h5_path = h5_path
        self.input_key = input_key
        self.condition_key = condition_key
        self.x_key = x_key
        self.y_key = y_key
        self.z_key = z_key
        self.dtype = dtype

        self._h5: Optional[h5py.File] = None
        self._input = None
        self._cond = None
        self._xp = None
        self._yp = None
        self._zp = None
        self._length = None
        self._pos_cache: Optional[torch.Tensor] = None  # cached (5160,3) on CPU

    def _ensure_open(self):
        """Lazy-open HDF5 file (safe with DataLoader workers)."""
        if self._h5 is None:
            self._h5 = h5py.File(self.h5_path, "r")
            self._input = self._h5[self.input_key]
            self._cond  = self._h5[self.condition_key]
            self._xp    = self._h5[self.x_key]
            self._yp    = self._h5[self.y_key]
            self._zp    = self._h5[self.z_key]
            self._length = self._input.shape[0]

    def __len__(self) -> int:
        self._ensure_open()
        return self._length

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        self._ensure_open()
        x_np  = self._input[idx]                            # (2,5160)
        c_np  = self._cond[idx]                             # (6,)

        # Cache shared positions once (as CPU tensor)
        if self._pos_cache is None:
            pos_np = _np_stack3(self._xp, self._yp, self._zp)  # (5160,3)
            self._pos_cache = torch.as_tensor(pos_np, dtype=self.dtype)

        x   = torch.as_tensor(x_np, dtype=self.dtype)            # (2,5160)
        pos = self._pos_cache                                    # (5160,3) shared
        c   = torch.as_tensor(c_np, dtype=self.dtype)            # (6,)
        return {"x": x, "pos": pos, "c": c, "idx": idx}

    def close(self):
        """Close file handle manually if needed."""
        if self._h5 is not None:
            try:
                self._h5.close()
            finally:
                self._h5 = None


# ----------------------------
# Embeddings, AdaLN-Zero
# ----------------------------
def sinusoidal_timestep_embedding(t: torch.Tensor, dim: int) -> torch.Tensor:
    """
    Sinusoidal embedding for scalar timesteps in [0,1] or scaled ints.
    t: (B,)
    returns: (B, dim)
    """
    device = t.device
    half = dim // 2
    # Log-scale frequencies for stability
    freqs = torch.exp(torch.linspace(math.log(1.0), math.log(10000.0), steps=half, device=device))
    args = t[:, None] * freqs[None, :]
    emb = torch.cat([torch.sin(args), torch.cos(args)], dim=-1)
    if dim % 2 == 1:
        emb = torch.nn.functional.pad(emb, (0, 1))
    return emb

class MLP(nn.Module):
    """Two-layer MLP with GELU and optional dropout."""
    def __init__(self, in_dim, hidden, out_dim, dropout=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, out_dim),
        )
    def forward(self, x):
        return self.net(x)

class AdaLNZero(nn.Module):
    """
    AdaLN-Zero: layer-norm then FiLM-like mod with zero-initialized scale/shift/gate.
    Given x (B,L,H) and cond_vec (B,C), predict gamma,beta,gate in (B,H) then:
      y = LN(x) * (1+gamma) + beta
      out = x + gate * F(y)
    """
    def __init__(self, hidden_dim: int, cond_dim: int):
        super().__init__()
        self.ln = nn.LayerNorm(hidden_dim, elementwise_affine=False)
        self.mod = nn.Linear(cond_dim, hidden_dim * 3)
        nn.init.zeros_(self.mod.weight)
        nn.init.zeros_(self.mod.bias)

    def forward(self, x: torch.Tensor, cond_vec: torch.Tensor):
        h = self.ln(x)                        # (B,L,H)
        m = self.mod(cond_vec)                # (B,3H)
        gamma, beta, gate = torch.chunk(m, 3, dim=-1)  # (B,H) each
        gamma = gamma.unsqueeze(1)            # (B,1,H)
        beta  = beta.unsqueeze(1)
        gate  = gate.unsqueeze(1)
        y = h * (1 + gamma) + beta
        return y, gate


# ----------------------------
# Transformer Block (DiT-style)
# ----------------------------
class DiTBlock(nn.Module):
    """Transformer block with AdaLN-Zero conditioning on both Attn and MLP paths."""
    def __init__(self, hidden_dim: int, n_heads: int, mlp_ratio: float = 4.0, dropout: float = 0.0, cond_dim: int = 256):
        super().__init__()
        self.attn_mod = AdaLNZero(hidden_dim, cond_dim)
        self.mlp_mod  = AdaLNZero(hidden_dim, cond_dim)

        self.attn = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=n_heads,
                                          batch_first=True, dropout=dropout)
        self.attn_drop = nn.Dropout(dropout)

        mlp_hidden = int(hidden_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden, hidden_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor, cond_vec: torch.Tensor):
        # Attention path
        x_mod, gate_attn = self.attn_mod(x, cond_vec)
        attn_out, _ = self.attn(x_mod, x_mod, x_mod, need_weights=False)  # (B,L,H)
        x = x + gate_attn * self.attn_drop(attn_out)

        # MLP path
        x_mod, gate_mlp = self.mlp_mod(x, cond_vec)
        x = x + gate_mlp * self.mlp(x_mod)
        return x


# ----------------------------
# PMT DiT Model
# ----------------------------
class PMTDiT(nn.Module):
    """
    DiT-style transformer conditioned on timestep + condition vector.
    - signal_mlp: (npe,time)->h
    - pos_mlp:    (x,y,z)->h
    - token = signal_emb + pos_emb
    - blocks: DiT blocks with AdaLN-Zero
    - head: predict eps for (npe,time)
    """
    def __init__(self,
                 hidden_dim: int = 256,
                 depth: int = 6,
                 n_heads: int = 8,
                 cond_in_dim: int = 6,
                 t_embed_dim: int = 256,
                 cond_embed_dim: int = 256,
                 dropout: float = 0.0):
        super().__init__()
        self.hidden_dim = hidden_dim

        # Embeddings
        self.signal_mlp = MLP(in_dim=2, hidden=hidden_dim, out_dim=hidden_dim, dropout=dropout)
        self.pos_mlp    = MLP(in_dim=3, hidden=hidden_dim, out_dim=hidden_dim, dropout=dropout)

        # Timestep & condition embeddings
        self.t_embed_dim = t_embed_dim
        self.c_embed = MLP(in_dim=cond_in_dim, hidden=cond_embed_dim, out_dim=t_embed_dim, dropout=dropout)
        self.fuse = MLP(in_dim=t_embed_dim, hidden=t_embed_dim, out_dim=t_embed_dim, dropout=dropout)  # fuse t+c

        # Transformer blocks
        self.blocks = nn.ModuleList([
            DiTBlock(hidden_dim=hidden_dim, n_heads=n_heads, mlp_ratio=4.0, dropout=dropout, cond_dim=t_embed_dim)
            for _ in range(depth)
        ])

        # Head
        self.norm = nn.LayerNorm(hidden_dim)
        self.head = nn.Linear(hidden_dim, 2)  # predict eps for (npe,time)

    def forward(self, x_sig: torch.Tensor, pos: torch.Tensor, c: torch.Tensor, t_float01: torch.Tensor):
        """
        x_sig:     (B, 2, 5160)
        pos:       (B, 5160, 3)
        c:         (B, 6)
        t_float01: (B,) float in [0,1] for time embedding
        returns:   eps_pred (B, 2, 5160)
        """
        # Tokenize
        sig_tok = self.signal_mlp(x_sig.transpose(1, 2))  # (B,5160,2)->(B,5160,h)
        pos_tok = self.pos_mlp(pos)                       # (B,5160,3)->(B,5160,h)
        tok = sig_tok + pos_tok                           # (B,5160,h)

        # DiT conditioning vector
        t_emb = sinusoidal_timestep_embedding(t_float01, self.t_embed_dim)  # (B,D)
        c_emb = self.c_embed(c)                                             # (B,D)
        cond_vec = self.fuse(t_emb + c_emb)                                 # (B,D)

        # Transformer
        h = tok
        for blk in self.blocks:
            h = blk(h, cond_vec)                                            # (B,5160,h)

        h = self.norm(h)
        eps = self.head(h)             # (B,5160,2)
        return eps.transpose(1, 2)     # (B,2,5160)


# ----------------------------
# DDPM schedules and q-sample
# ----------------------------
def make_beta_schedule_linear(T: int, start: float = 1e-4, end: float = 2e-2):
    """Linear beta schedule in [start, end]."""
    betas = torch.linspace(start, end, T, dtype=torch.float32)
    alphas = 1.0 - betas
    alpha_bars = torch.cumprod(alphas, dim=0)  # (T,)
    return betas, alphas, alpha_bars

def make_beta_schedule_cosine(T: int, s: float = 0.008):
    """
    Cosine schedule (Nichol & Dhariwal, 2021):
      alpha_bar(t) = cos^2( (t/T + s)/(1+s) * pi/2 )
    Convert to discrete betas via alpha_bar ratio.
    """
    steps = torch.arange(T + 1, dtype=torch.float64)
    f = torch.cos(((steps / T) + s) / (1 + s) * math.pi / 2) ** 2  # (T+1,)
    f = f / f[0]  # normalize so alpha_bar(0)=1
    # beta_t = 1 - alpha_bar(t+1)/alpha_bar(t)
    betas = (1 - (f[1:] / f[:-1]).clamp(min=1e-12)).to(torch.float32)
    betas = betas.clamp(min=1e-8, max=0.999)
    alphas = 1.0 - betas
    alpha_bars = torch.cumprod(alphas, dim=0).to(torch.float32)
    return betas, alphas, alpha_bars

def q_sample(x0: torch.Tensor, t_int: torch.Tensor, alpha_bars: torch.Tensor):
    """
    DDPM forward noising:
      x_t = sqrt(alpha_bar[t]) * x0 + sqrt(1 - alpha_bar[t]) * eps
    x0:      (B, 2, 5160)
    t_int:   (B,) integer in [0, T-1]
    returns: x_t, eps
    """
    B = x0.shape[0]
    eps = torch.randn_like(x0)
    ab = alpha_bars.to(x0.device)[t_int].view(B, 1, 1)  # (B,1,1)
    x_t = torch.sqrt(ab) * x0 + torch.sqrt(1.0 - ab) * eps
    return x_t, eps


# ----------------------------
# Training step
# ----------------------------
def training_step_ddpm(model: PMTDiT, batch: Dict[str, torch.Tensor], optimizer, device,
                       alpha_bars: torch.Tensor, T: int):
    """
    One DDPM training step:
      - sample t_int in [0, T-1]
      - q_sample to get (x_t, eps)
      - predict eps and compute MSE
    """
    model.train()
    x0  = batch["x"].to(device)     # (B,2,5160)
    pos = batch["pos"].to(device)   # (B,5160,3)
    c   = batch["c"].to(device)     # (B,6)
    B   = x0.shape[0]

    t_int = torch.randint(0, T, (B,), device=device)          # (B,)
    x_t, eps = q_sample(x0, t_int, alpha_bars)

    # Normalized timestep [0,1] for embedding
    t_norm = t_int.float() / (T - 1)

    eps_pred = model(x_sig=x_t, pos=pos, c=c, t_float01=t_norm)  # (B,2,5160)
    loss = torch.mean((eps_pred - eps) ** 2)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    return loss.item()


# ----------------------------
# Minimal runnable demo
# ----------------------------
if __name__ == "__main__":
    import argparse
    import os

    # --- CLI ---
    parser = argparse.ArgumentParser(description="PMT DiT diffusion demo (DDPM q-sample).")
    parser.add_argument("-p", "--path", required=True, help="Path to HDF5 file")
    parser.add_argument("--batch-size", type=int, default=4)
    parser.add_argument("--hidden-dim", type=int, default=256)
    parser.add_argument("--depth", type=int, default=6)
    parser.add_argument("--heads", type=int, default=8)
    parser.add_argument("--dropout", type=float, default=0.0)
    parser.add_argument("--schedule", choices=["linear", "cosine"], default="cosine")
    parser.add_argument("--T", type=int, default=1000, help="Number of diffusion steps")
    parser.add_argument("--lr", type=float, default=1e-4)
    parser.add_argument("--steps", type=int, default=1, help="Number of demo training steps to run")
    parser.add_argument("--workers", type=int, default=4)
    args = parser.parse_args()

    # --- Device & seed ---
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.manual_seed(42)
    if device == "cuda":
        torch.cuda.manual_seed_all(42)

    # --- Dataset / Loader ---
    ds = PMTH5Dataset(h5_path=args.path)
    loader = DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.workers,
        pin_memory=(device == "cuda"),
        persistent_workers=(args.workers > 0),
    )

    # --- Model / Optim ---
    model = PMTDiT(
        hidden_dim=args.hidden_dim,
        depth=args.depth,
        n_heads=args.heads,
        cond_in_dim=6,
        t_embed_dim=args.hidden_dim,      # often set equal to hidden_dim
        cond_embed_dim=args.hidden_dim,   # dit-style
        dropout=args.dropout,
    ).to(device)
    optim = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-2)

    # --- Beta schedule ---
    if args.schedule == "linear":
        _, _, alpha_bars = make_beta_schedule_linear(args.T)
    else:
        _, _, alpha_bars = make_beta_schedule_cosine(args.T)
    alpha_bars = alpha_bars.to(device)

    # --- Run demo steps ---
    it = iter(loader)
    for step in range(args.steps):
        try:
            batch = next(it)
        except StopIteration:
            it = iter(loader)
            batch = next(it)

        loss = training_step_ddpm(model, batch, optim, device, alpha_bars, args.T)
        print(f"[step {step+1}/{args.steps}] loss: {loss:.6f}")

    print("Done.")

usage: ipykernel_launcher.py [-h] -p PATH [--batch-size BATCH_SIZE]
                             [--hidden-dim HIDDEN_DIM] [--depth DEPTH]
                             [--heads HEADS] [--dropout DROPOUT]
                             [--schedule {linear,cosine}] [--T T] [--lr LR]
                             [--steps STEPS] [--workers WORKERS]
ipykernel_launcher.py: error: the following arguments are required: -p/--path


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import torch as th
from model import DiT3D
import numexpr as ne
ne.set_num_threads(64)     # 또는 더 작게: 16, 32 등
model = DiT3D(
    geom_csv="/home/work/GENESIS/GENESIS-pmj0324/GENESIS/genesis/utils/detector_geometry.csv",  # (x,y,z) 5160개
    L=5160, C=2,
    d_model=64, depth=4, nhead=8,
    te_dim=64, ff_mult=4,
    num_freq=16, sigma=50.0,
    cond_dim=0
).cuda()

Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

In [2]:
import h5py, torch as th
from torch.utils.data import DataLoader, Dataset

class LinearSchedule:
    def __init__(self, T=1000, beta_start=1e-4, beta_end=2e-2, device="cuda"):
        self.T = T
        beta = th.linspace(beta_start, beta_end, T, device=device)
        self.alpha = 1.0 - beta
        self.alphabar = th.cumprod(self.alpha, dim=0)  # (T,)

    def add_noise(self, x0, t, eps=None):
        if eps is None: eps = th.randn_like(x0)
        a_bar = self.alphabar[t].view(-1, 1, 1)       # (B,1,1)
        x_t = th.sqrt(a_bar) * x0 + th.sqrt(1.0 - a_bar) * eps
        return x_t, eps

class H5InputSet(Dataset):
    def __init__(self, h5_path, key="input"):
        self.f = h5py.File(h5_path, "r")
        self.ds = self.f[key]            # (N,2,5160)
        self.N  = self.ds.shape[0]
    def __len__(self): return self.N
    def __getitem__(self, i):
        x = th.from_numpy(self.ds[i]).float()  # (2,5160)
        x = x.transpose(0,1).contiguous()      # -> (5160,2)
        return x
    def close(self): self.f.close()

In [3]:
ds = H5InputSet("/home/work/GENESIS/GENESIS-data/minje-version/22645_0730-noinf.h5")
dl = DataLoader(ds, batch_size=8, shuffle=True, num_workers=0, pin_memory=True)

In [4]:
model.train()
opt = th.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
sched = LinearSchedule(T=1000, device="cuda")

In [5]:
for epoch in range(5):
    total=steps=0
    for x0 in dl:
        x0 = x0.cuda(non_blocking=True)                   # (B,5160,2)
        B = x0.size(0)
        t = th.randint(0, sched.T, (B,), device="cuda")
        print(t)
        x_t, eps = sched.add_noise(x0, t)
        eps_hat = model(x_t, t)
        loss = ((eps_hat - eps) ** 2).mean()
        opt.zero_grad(set_to_none=True)
        loss.backward()
        th.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        total += loss.item(); steps += 1
    print(f"[Epoch {epoch+1}] loss={total/steps:.6f}")

tensor([743, 266,  79, 281, 515, 209, 242, 917], device='cuda:0')
tensor([589,  87, 740, 467, 855, 630, 785, 771], device='cuda:0')
tensor([550, 838, 897, 997, 316,  35, 796, 119], device='cuda:0')
tensor([  1, 560, 183, 460, 864, 704, 123, 809], device='cuda:0')
tensor([872, 610, 712, 106, 759, 760, 582, 120], device='cuda:0')
tensor([220,  19, 484, 800, 309, 787, 909, 785], device='cuda:0')
tensor([978, 491, 684,   6, 745, 721, 549, 686], device='cuda:0')
tensor([ 16, 986, 516, 184, 553, 112, 677, 889], device='cuda:0')
tensor([281, 618, 794, 275, 244, 468, 454, 151], device='cuda:0')
tensor([517,  22, 872, 607, 305, 299, 876, 375], device='cuda:0')
tensor([366, 961,  46,  15, 330, 277, 582, 397], device='cuda:0')
tensor([823,  74, 606, 499, 449, 958,  28, 656], device='cuda:0')
tensor([743, 930, 416, 788, 682, 302, 674, 462], device='cuda:0')
tensor([236, 715, 269,  55, 407, 417, 487, 546], device='cuda:0')
tensor([788, 996,  32, 152, 625, 723, 489, 912], device='cuda:0')
tensor([48

KeyboardInterrupt: 